In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import os.path
sc = SparkContext.getOrCreate()

In [ ]:
os.chdir('C:/Users/flyin/Dropbox/AW Inspur/Recommender/ml-latest-small')
my_path = os.path.abspath(os.path.dirname('__file__'))

In [ ]:
lines = spark.read.csv(os.path.join(my_path, "ratings.csv")).rdd

In [ ]:
(os.path.join(my_path, "ratings.csv"))

In [ ]:
lines

In [ ]:
header = lines.first()

In [ ]:
# remove header

new_lines = lines.filter(lambda line: line != header)

In [ ]:
# generate new RDD with column names
ratingsRDD = new_lines.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),rating=float(p[2]), timestamp=int(p[3])))

In [ ]:
# create dataframe from RDD
# generate training and test dataframe
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [ ]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

In [ ]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

In [ ]:
print(userRecs.show(2,truncate= False))